# Parameter references

Building on the previous discussion about the dual roles of a `Parameter` - as both a value holder and a metadata container - let's explore how parameters can go a step further by acting as dynamic references to other parameters. When configured with `allow_refs=True`, a `Parameter` can serve as a live link to another `Parameter`, mirroring its current value. This capability enables more intricate relationships between parameters, allowing for automatic value synchronization and forming the basis for reactive programming.

In [ ]:
import param

class U(param.Parameterized):
    
    a = param.Number()
    
class V(param.Parameterized):
    
    b = param.Number(default=None, allow_refs=True)

u = U(a=3.14)
v = V(b=u.param.a)

v.b

By declaring that `V.b` allows references we have made it possible to pass the Parameter `U.a`, which means `v.b` will reflect the value of `u.a`:

In [ ]:
u.a = 1.57

v.b

This unidirectional link will be in effect until something else tries to set the value:

In [ ]:
v.b = 14.1
u.a = 13.2

v.b

In other words, if the value is overridden from the outside the link will be automatically removed.

Simple references are resolved when `allow_refs=True` but to allow nested references we separately have to set `nested_refs=True`.

In [ ]:
class W(V):
    
    c = param.List(allow_refs=True, nested_refs=True)
    
u1 = U(a=3)
u2 = U(a=13)

w = W(c=[u1.param.a, u2.param.a]) 

w.c

When we modify either `u1.a` or `u2.a`, `w.c` will update:

In [ ]:
u1.a = 7

w.c

Note that `Parameter` types are not the only types of valid references. The full list of valid references include:

- Class and instance `Parameter` objects
- Functions or methods annotated with `param.depends`
- Functions wrapped with `param.bind`
- Reactive expressions declared using `param.rx`
- Asynchronous generators
- Custom objects transformed into a valid reference with a hook registered with `param.parameterized.register_reference_transform`.

There are two utility functions which allow resolving all parameters a reference depends on and the current value of the reference:

In [ ]:
from param.parameterized import resolve_ref, resolve_value

resolve_ref(u1.param.a), resolve_value(u1.param.a)

## Skipping Reference Updates

Since references are resolved eagerly whenever one of the dependencies change we may run into situations where we want to control when a reference is updated. Specifically we may want to skip resolving a reference if one of the inputs does not meet some condition or only if a certain event is triggered.

Let's see how we can configure this. Here we will create a class `W` with parameters `a` and `b` and a `run` event. We then define a function to `add` parameters `a` and `b` but only if the `run` event is active. To do this we can raise a `param.Skip()` exception in the function.

In [ ]:
class W(param.Parameterized):
    
    a = param.Number()
    
    b = param.Number()

    run = param.Event()

w = W(a=0, b=2)

def add(a, b, run):
    if not run:
        raise param.Skip()
    return a + b

We can now bind all three parameters to the function:

In [ ]:
v = V(b=param.bind(add, w.param.a, w.param.b, w.param.run))

v.b

Even though we initialized `v.b` with a reference it will not resolve this reference until we trigger a `run` event:

In [ ]:
w.param.trigger('run')

v.b